# 2A - DATA CLEANING:

#### Prop 64 - Allows marijuana growth near schools and parks. 

In the following notebook, I will be examining the Los Angeles data city catalog's [Arrests from 2010 to Present](https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4) csv file which contains roughly 1.2 million arrests. The dataset is provided by the Los Angeles Police Department (LAPD). Each row is the booking of an arrestee, and each column contains details about the arrest incident. For the purpose of this project, I will only examine marijuana related arrests by applying a filter to the `Charge Description` if it contains the word 'marijuana'. 

I will also calculate the distance between every marijuana arrest and dispensary, and examine the number of marijuana arrests within 0.5 miles of a dispensary over each year. 

I will also calcualte the distance between every marijuana arrest and Los Angeles Unified School District (LAUSD) school, and examine the number of marijuana arrests within 0.5 miles of a school over each year. 

#### Arrests Data Dictionary:

| Column Name | Type | Description |
| --- | --- | --- | 
| Report ID | int64 | ID for the Arrest | 
| Arrest Date |  DateTime | YYYY/MM/DD |
| Time | float64 | 24 hour military time |
| Area ID |  int64 | 21 LAPD stations referred to as Geographic Areas that are sequentially numbered from 1-21 |
| Area Name | object  | Area ID's name designation that references a landmark or the surrounding community that an LAPD station is responsible for|
| Reporting District | int64 |  A four-digit code that represents a sub-area within a Geographic Area | 
| Age | int64 | Age of the arrestee | 
| Sex Code | object| F - Female, M - Male |
| Descent Code | object | Arrestee's descent code |
| Charge Group Code | object | Category of arrest charge |
| Charge Group Description | object | Defines the charge provided | 
| Arrest Type Code | object | A code to indicate the type of charge the individual was arrested for. D - Dependent F - Felony I - Infraction M - Misdemeanor O - Other |
| Charge | object | The charge the individual was arrested for |
| Charge  Description | object | Defines the Charge provided |
| Address | object | Street address of crime incident |
| Location | object  | The location where the crime incident occurred. XY coordinates reflect the nearest 100 block |
| disp_0.5_mile | int64 | Number of dispensaries within 0.5 miles of each arrest | 
| disp_1_mile | int64 | Number of dispensaries within 1 mile of each arrest | 
| school_0.5_mile | int64 | Number of school(s) within 0.5 miles of each arrest | 
| school_0.5_mile | int64 | Number of school(s) within 1 mile of each arrest | 


#### Marijuana Dispensary  Data Dictionary:

| Column Name | Type | Description | 
| --- | --- | --- |
| name | object | Dispensary name|
| is_closed | boolean | Indicates whether a dispensary is closed or open |
| url| object | Yelp dispensary website link | 
| lat | float | Dispensary's latitude | 
| long | float | Dispensary's longitude | 
| location | object | Dispensary's coordinates| 


#### LAUSD Data Dictionary:

| Column Name | Type | Description | 
| --- | --- | --- |
| School | object | School name |
| Zip Code | int64 | School's zip code |
| Grades | object | Grades offered in school |
| Local District | object | School's local district | 
| Board District | object | School's board district | 
| complete_address | object | School's address |
| lat | float | School's latitude | 
| long | float | School's longitude | 
| location | object | School's coordinates| 

## Data Clean Up:

In [211]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from geopy.distance import geodesic

### **Marijuana Dispensary Clean Up**

I will first clean up the marijuana dispensary csv file that was queried from Yelp.

In [114]:
df_weed = pd.read_csv('../data/df_weed.csv')
df_weed.head()

,Unnamed: 0,name,is_closed,url,lat,long,location
0,0,Extra Special Delivery,False,https://www.yelp.com/biz/extra-special-deliver...,34.166720,-118.375185,"(34.1667203, -118.3751849)"
1,1,Ganjarunner,False,https://www.yelp.com/biz/ganjarunner-los-angel...,34.101610,-118.302060,"(34.10161, -118.30206)"
2,2,The Higher Path,False,https://www.yelp.com/biz/the-higher-path-sherm...,34.149339,-118.439875,"(34.1493390052598, -118.439874686508)"
3,3,Kushfly,False,https://www.yelp.com/biz/kushfly-los-angeles-2...,34.127600,-118.346690,"(34.1276, -118.34669)"
4,4,MedMen West Hollywood,False,https://www.yelp.com/biz/medmen-west-hollywood...,34.090591,-118.367290,"(34.0905911417357, -118.367290442404)"


There are a total of 162 dispensaries in Los Angeles, and none of them are closed:

In [116]:
len(df_weed)

162

In [115]:
df_weed['is_closed'].value_counts()

False    162
Name: is_closed, dtype: int64

The `is_closed` variable does not provide any valuable information, and the `location` variable is redundant because of the `lat` and `long` variables.
Furthermore, the `url` column is not needed for our exploratory data analysis purposes, so the three variables will be dropped.

In [120]:
df_weed = df_weed[['name','lat','long']]
df_weed.head()

,name,lat,long
0,Extra Special Delivery,34.166720,-118.375185
1,Ganjarunner,34.101610,-118.302060
2,The Higher Path,34.149339,-118.439875
3,Kushfly,34.127600,-118.346690
4,MedMen West Hollywood,34.090591,-118.367290


In [119]:
df_weed.isnull().sum()

name    0
lat     0
long    0
dtype: int64

There are no null values in the `df_weed` dataframe.

### **Arrests Data**

I will now clean up the following arrests dataframe

In [12]:
arrests = pd.read_csv('../data/Arrest_Data_from_2010_to_Present.csv')
arrests.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,111421932,07/02/2016,1845.0,14,Pacific,1411,50,M,H,24.0,Miscellaneous Other Violations,M,LAMC,LOS ANGELES MUNICIPAL CODE,BROOKS,OCEAN FRONT,"(33.9918, -118.4791)"
1,121801235,12/18/2016,940.0,18,Southeast,1822,38,F,B,24.0,Miscellaneous Other Violations,M,LAMC,LOS ANGELES MUNICIPAL CODE,300 W CENTURY BL,NaN,"(33.9456, -118.2784)"
2,150604240,01/03/2016,1315.0,6,Hollywood,669,43,M,O,24.0,Miscellaneous Other Violations,M,71.02LAMC,HIRE VEH W/O LIC,HOBART,SANTA MONICA,"(34.0908, -118.3046)"
3,150704165,04/27/2016,2230.0,7,Wilshire,721,20,F,B,6.0,Larceny,M,484(A)PC,GRAND THEFT (OVER $400),8500 BEVERLY BL,NaN,"(34.0761, -118.3766)"
4,151405507,01/22/2016,1309.0,14,Pacific,1427,60,M,W,17.0,Liquor Laws,M,25620BP,OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE,VENICE,MOTOR,"(34.0237, -118.4246)"


I am going to convert the `Arrest Date` to DateTime for better processing.

In [18]:
arrests['Arrest Date'] = pd.to_datetime(arrests['Arrest Date'])

I am only interested in arrests that occured from January 1st, 2015 until April 3rd, 2019.

In [21]:
arrests = arrests.sort_values('Arrest Date')
arrests = arrests[arrests['Arrest Date'] >= '2015-01-01']

I am now going to split the `Location` variable into separate `Lat` and `Long` variables for two purposes:
1. Plotting the coordinates of each arrest
2. Calculating the distance between each arrest and each dispensary or each school

In [25]:
# Using regex to re-configure Location variable
arrests['Location'] = arrests['Location'].map(lambda x: re.sub('[(),°]', '', x)).str.split()

In [64]:
# Create latitude and longitude variables:
arrests['lat'] = arrests['Location'].map(lambda x: x[0])
arrests['long'] = arrests['Location'].map(lambda x: x[1])

# Converting the latitude and longitude variables to floats
arrests['lat'] = arrests['lat'].map(lambda x: float(x))
arrests['long'] = arrests['long'].map(lambda x: float(x))

# Convert charge description variable into a string and lowercase
arrests['Charge Description'] = arrests['Charge Description'].map(lambda x: str(x))
arrests['Charge Description'] = arrests['Charge Description'].map(lambda x: x.lower())

arrests.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,lat,long,marijuana_related
816415,4196571,2015-01-01,110.0,14,Pacific,1457,21,M,W,22.0,Driving Under Influence,M,23152(A)VC,drunk driving alcohol/drugs,CULVER,STONER,"[33.9962, -118.4167]",33.9962,-118.4167,NaN
816412,4196538,2015-01-01,10.0,9,Van Nuys,943,34,M,H,24.0,Miscellaneous Other Violations,M,853.7PC,fta after written promise,VAN NUYS BL,AETNA ST,"[34.1802, -118.4487]",34.1802,-118.4487,NaN
816414,4196555,2015-01-01,15.0,6,Hollywood,646,33,M,W,24.0,Miscellaneous Other Violations,F,3000.08CPC,parole warrant,WILCOX AV,SUNSET BL,"[34.098, -118.331]",34.0980,-118.3310,NaN
816413,4196545,2015-01-01,30.0,9,Van Nuys,966,50,M,W,22.0,Driving Under Influence,M,23152(A)VC,drunk driving alcohol/drugs,WOODMAN AV,RIVERSIDE,"[34.1576, -118.4312]",34.1576,-118.4312,NaN
816452,4196699,2015-01-01,315.0,7,Wilshire,723,44,M,O,22.0,Driving Under Influence,M,23152(A)VC,drunk driving alcohol/drugs,BEVERLY,LAUREL,"[34.0761, -118.3648]",34.0761,-118.3648,NaN


In [65]:
arrests.shape

(477440, 20)

There are 477,440 arrests from 2015 to 2019. I am now going to extract marijuana-related arrests from the dataframe because the purpose of the problem statement is to examine the relationship between marijuana growth and schools.

In [66]:
charge_list = list(arrests['Charge Description'].value_counts().index.sort_values())
charge_list = [x.lower() for x in charge_list]

There are 32 different kind of marijuana charges that deal with cultivating, possessing, selling, smoking in public, and dealing with minors:

In [67]:
len([x for x in charge_list if 'mari' in x])

32

In [68]:
green_charge = [x for x in charge_list if 'mari' in x]
green_charge

['attempt - sell/furnish/etc marijuana',
 'cultivate >6 marij plants viol envrnt law',
 'cultivating <6 marijuana plants',
 'furnishing marijuana to minor over 14 yrs',
 'give/transport/etc < 28.5 grams marijuana',
 'induce/etc minor to use/sell marijuana',
 'minor poss 28.5+ grams marijuana/school',
 'minor poss < 28.5 grams marijuana/school',
 'poss for sale of marijuana to a minor',
 'poss marijuana or concentrated cannabis',
 'poss of more than 28.5 grams of marijuana',
 'poss open cont/packg marij drivr/passnger',
 'poss/sale marij ovr 21 employ per 20/belw',
 'poss/smoke/ingest marij school/daycare/yc',
 'possess 28.5 grams or less of marijuana',
 'possess 28.5 grams or less of marijuana**',
 'possess marijuana for sale',
 'possess marijuana for sale under age 18',
 'possess of marijuana while driving veh',
 'possession marijuana for sale',
 'possession of marijuana in school',
 'sale/offer to sell/transport marijuana',
 'sale/trans >28.5g marijuana or >4g',
 'sale/transport mari

In [83]:
# Creating a marijuana_related column that lists charges if only they are related to marijuana
arrests['marijuana_related'] = arrests['Charge Description'].map(lambda x: x if x in green_charge else np.NaN)

There have been 4156 marijuana related arrests between 2015 and 2019.

In [76]:
len(arrests[~arrests['marijuana_related'].isnull()])

4156

In [85]:
arrests = arrests[~arrests['marijuana_related'].isnull()]

In [86]:
arrests.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,lat,long,marijuana_related
811489,151804330,2015-01-01,1610.0,18,Southeast,1822,19,M,B,16.0,Narcotic Drug Laws,M,11357BHS,possess 28.5 grams or less of marijuana**,MAIN ST,CENTURY BL,"[33.9456, -118.2739]",33.9456,-118.2739,possess 28.5 grams or less of marijuana**
816495,4196822,2015-01-01,820.0,6,Hollywood,645,23,M,B,16.0,Narcotic Drug Laws,F,11360(A)HS,transport/sell/furnish/etc marijuana,6800 HOLLYWOOD BL,NaN,"[34.1016, -118.3387]",34.1016,-118.3387,transport/sell/furnish/etc marijuana
816686,4197433,2015-01-02,1030.0,14,Pacific,1457,24,F,W,16.0,Narcotic Drug Laws,F,11360(A)HS,transport/sell/furnish/etc marijuana,12300 CULVER BL,NaN,"[33.992, -118.4201]",33.9920,-118.4201,transport/sell/furnish/etc marijuana
816723,4197611,2015-01-02,1530.0,14,Pacific,1494,30,M,O,16.0,Narcotic Drug Laws,F,11359HS,possession marijuana for sale,300 WORLD WY,NaN,"[33.944, -118.4073]",33.9440,-118.4073,possession marijuana for sale
816982,4198423,2015-01-03,1940.0,3,Southwest,311,26,M,H,16.0,Narcotic Drug Laws,F,11360(A)HS,transport/sell/furnish/etc marijuana,JEFFERSON,COCHRAN,"[34.026, -118.3652]",34.0260,-118.3652,transport/sell/furnish/etc marijuana


### LAUSD Data Clean Up

I will now clean up the LAUSD dataframe from their [website](http://www.lausd.k12.ca.us/lausd/offices/bulletins/).

In [123]:
LA_pub_school = pd.read_csv('http://www.lausd.k12.ca.us/lausd/offices/bulletins/lausdk12.tab',sep='\t')
LA_pub_school.head()

,Cost Center Code,Legacy Code,School,Address,City,State,Zip Code,Telephone,Fax,Grades,Calendar,Local District,Board District,File Build Date
0,1585701,5857,107th Street Elementary,147 E 107th St,Los Angeles,CA,90003,323-756-8137,323-779-6942,K- 5,1 Trk,S,7-Dr Richard A Vladovic,2019-04-27
1,1585702,5858,107th Street Elementary Science/Tech/Eng/Arts/...,147 E 107th St,Los Angeles,CA,90003,323-756-8137,323-779-6942,1- 5,1 Trk,S,7-Dr Richard A Vladovic,2019-04-27
2,1583601,5836,109th Street Elementary,10915 S Mc Kinley Ave,Los Angeles,CA,90059,323-756-9206,323-755-2307,K- 5,1 Trk,S,7-Dr Richard A Vladovic,2019-04-27
3,1708201,7082,10th Street Elementary,1000 Grattan St,Los Angeles,CA,90015,213-380-8990,213-480-6732,1- 5,1 Trk,C,2-Monica Garcia,2019-04-27
4,1588401,5884,112th Street Elementary,1265 E 112th St,Los Angeles,CA,90059,323-567-2108,323-567-2611,K- 5,1 Trk,S,7-Dr Richard A Vladovic,2019-04-27


I am going to combine the `Address`, `City`, `State`, `Zip Code` columns to create a `complete_address` variable which will then be used to obtain the coordinates for each school.

In [124]:
# combining addresses to get coordinates
LA_pub_school['complete_address'] = LA_pub_school['Address'] +' '+ LA_pub_school['City'] +' '+ LA_pub_school['State'] +' '+ LA_pub_school['Zip Code'].astype(str)
LA_pub_school['complete_address'] = LA_pub_school['complete_address'].astype(str)

In [127]:
# Examining File Build Date
LA_pub_school['File Build Date'].value_counts()

2019-04-27    1331
Name: File Build Date, dtype: int64

The `File Build Date` is all the same, so I am going to drop the variable. I am also going to drop the individual address columns, the `Cost Center Code`, `Legacy Code`, `Telephone`, `Fax`, and `Calendar`.

In [128]:
LA_pub_school = LA_pub_school.drop(['Address','City','State',
                                    'Cost Center Code', 'Legacy Code',
                                    'Telephone','Fax','Calendar','File Build Date'], 1)
LA_pub_school.head()

,School,Zip Code,Grades,Local District,Board District,complete_address
0,107th Street Elementary,90003,K- 5,S,7-Dr Richard A Vladovic,147 E 107th St Los Angeles CA 90003
1,107th Street Elementary Science/Tech/Eng/Arts/...,90003,1- 5,S,7-Dr Richard A Vladovic,147 E 107th St Los Angeles CA 90003
2,109th Street Elementary,90059,K- 5,S,7-Dr Richard A Vladovic,10915 S Mc Kinley Ave Los Angeles CA 90059
3,10th Street Elementary,90015,1- 5,C,2-Monica Garcia,1000 Grattan St Los Angeles CA 90015
4,112th Street Elementary,90059,K- 5,S,7-Dr Richard A Vladovic,1265 E 112th St Los Angeles CA 90059


I am now going to check for duplicate schools based on the `complete_address` and remove the duplicates. These schools may have different names or be separated into different schools such as `107th Street Elementary	` and `107th Street Elementary Science/Tech/Eng/Arts/...`.

In [166]:
LA_pub_school = LA_pub_school[~LA_pub_school.duplicated(subset='complete_address')].sort_values('complete_address')
LA_pub_school.reset_index(drop= True, inplace = True)
LA_pub_school.shape

(931, 6)

After removing duplicate schools There are 931 LAUSD schools. 

Obtaining each school's coordinates through googlemap's API:

In [183]:
import googlemaps
gmaps_key = googlemaps.Client(key = '##########') # API key blurred out

In [159]:
lat_list = []
long_list = []

for address in LA_pub_school['complete_address']:
    g = gmaps_key.geocode(address)
    
    try:
        lat = g[0]['geometry']['location']['lat']
        lng = g[0]['geometry']['location']['lng']
    except:
        lat = np.NaN
        lng = np.NaN
    
    lat_list.append(lat)
    long_list.append(lng)

In [172]:
LA_pub_school['lat'] = lat_list
LA_pub_school['long'] = long_list

In [174]:
LA_pub_school['coord'] = LA_pub_school['lat'].astype(str)+', '+ LA_pub_school['long'].astype(str) 

I am now going to remove duplicate schools based on their coordinates (`coord`).

In [198]:
LA_pub_school = LA_pub_school[~LA_pub_school.duplicated(subset='coord')]

In [199]:
LA_pub_school.reset_index(drop= True, inplace = True)
LA_pub_school.shape

(909, 9)

There are now 909 unique LAUSD schools.

In [201]:
LA_pub_school.head()

,School,Zip Code,Grades,Local District,Board District,complete_address,lat,long,coord
0,Frank Del Olmo Elementary,90004,K- 5,C,2-Monica Garcia,100 N New Hampshire Ave Los Angeles CA 90004,34.072925,-118.292591,"34.072925, -118.29259099999999"
1,10th Street Elementary,90015,1- 5,C,2-Monica Garcia,1000 Grattan St Los Angeles CA 90015,34.048317,-118.273235,"34.0483172, -118.2732352"
2,Elementary Community Day School,91311,K- 6,XS,3-Scott M Schmerelson,10001 Jumilla Ave Chatsworth CA 91311,34.251825,-118.563377,"34.2518249, -118.56337690000001"
3,Cesar E Chavez Learning Academy - Arts/Theatre...,91340,9-12,NE,6-Kelly Gonez,1001 Arroyo Ave San Fernando CA 91340,34.288514,-118.421011,"34.2885136, -118.42101100000001"
4,San Pedro Senior High Gifted STEAM Magnet,90731,9-11,S,7-Dr Richard A Vladovic,1001 W 15th St San Pedro CA 90731,33.730370,-118.299342,"33.73037, -118.2993419"


## Calculating Distance


### Distance between Arrests and Dispensaries

I will now calculate the distance in miles between each arrest and each dispensary.
There are 162 dispensaries and 4156 marijuana related arrests.

In [202]:
weed_distance = df_weed[['name','lat','long']]
weed_distance.head()

,name,lat,long
0,Extra Special Delivery,34.166720,-118.375185
1,Ganjarunner,34.101610,-118.302060
2,The Higher Path,34.149339,-118.439875
3,Kushfly,34.127600,-118.346690
4,MedMen West Hollywood,34.090591,-118.367290


In [204]:
weed_distance.shape

(162, 3)

In [205]:
arrest_distance = arrests[['Report ID', 'lat', 'long']]
arrest_distance.reset_index(drop=True, inplace= True)
arrest_distance.head()

,Report ID,lat,long
0,151804330,33.9456,-118.2739
1,4196822,34.1016,-118.3387
2,4197433,33.9920,-118.4201
3,4197611,33.9440,-118.4073
4,4198423,34.0260,-118.3652


In [206]:
arrest_distance.shape

(4156, 3)

The following function, `coord_dist`, calculates and returns the miles between two places based on their coordinates.

In [212]:
def coord_dist(disp_lat, disp_long, arr_lat, arr_long):
    dispensary_coord = (disp_lat, disp_long)
    arrest_coord = (arr_lat, arr_long)
    
    distance = geodesic(dispensary_coord, arrest_coord).miles
    return distance

The following nested for loop sequentially grabs a dispensary name, and then uses `coord_dist` to calculate the distances between the dispensary and each 4156 arrests. All 4156 distances are then appended to the `distance_list`. Afterwards, the `distance_list` is appended as a new column to the `arrest_distance` dataframe with the dispensary as the column name.

In [213]:
for disp_name, position in zip(list(weed_distance['name'].values),range(len(weed_distance))):

    distance_list = []

    for arrest in range(len(arrest_distance)):
        distance = coord_dist(
                              weed_distance[weed_distance['name']== disp_name]['lat'][position],
                              weed_distance[weed_distance['name']== disp_name]['long'][position], 
                              arrest_distance['lat'][arrest], 
                              arrest_distance['long'][arrest])

        distance_list.append(distance)

    arrest_distance[disp_name] = distance_list

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [222]:
arrest_distance.head()

,Report ID,lat,long,Extra Special Delivery,Ganjarunner,The Higher Path,Kushfly,MedMen West Hollywood,Green Earth Collective,HERB,...,EZ Custom Packaging,Giving Greens Delivery,Purecannastore,Spectwonders,Venice Medical Center,Pasadena 419,Venice Kush,The Blue Diamond Center,101 CBD,Bud Cellar
0,151804330,33.9456,-118.2739,16.310674,10.873636,16.966819,13.221308,11.340067,12.719135,6.883323,...,7.188795,2.401175,7.958150,5.016343,8.882261,16.573949,2.699480,14.978295,66.809728,16.215627
1,4196822,34.1016,-118.3387,4.951664,2.100821,6.667834,1.849670,1.806477,7.479557,6.465254,...,6.601689,10.160931,10.019831,15.738283,6.276706,12.944058,9.842293,3.699940,58.722302,4.861954
2,4197433,33.9920,-118.4201,12.314997,10.145962,10.903708,10.251097,7.440296,15.007006,10.376742,...,10.781956,9.853862,13.837747,13.894923,2.604547,20.327679,9.802252,12.083558,57.950214,12.199388
3,4197611,33.9440,-118.4073,15.461070,12.429242,14.275754,13.123732,10.361330,16.692006,11.329906,...,11.747321,9.278511,14.218789,12.074944,5.566106,21.709643,9.353913,15.014068,60.337654,15.346250
4,4198423,34.0260,-118.3652,9.716038,6.346375,9.518796,7.082776,4.453535,11.079990,6.707892,...,7.091525,7.527478,10.396720,12.588260,1.797437,16.414634,7.350535,8.992752,59.549391,9.605556


I will now create two columns, `disp_0.5_mile` and `disp_1_mile`, that will count the number of dispensaries that are within 0.5 miles and 1 mile respectively for each arrest. 

In [227]:
distance_arrest_dispensary = arrest_distance.drop(['Report ID','lat','long'], axis = 1).T
distance_arrest_dispensary.head()

,0,1,2,3,4,5,6,7,8,9,...,4146,4147,4148,4149,4150,4151,4152,4153,4154,4155
Extra Special Delivery,16.310674,4.951664,12.314997,15.461070,9.716038,9.036456,5.279060,6.460166,9.289508,11.247546,...,15.461070,15.478720,13.571459,15.613357,26.407741,15.478720,15.478720,13.571459,11.236899,1.639882
Ganjarunner,10.873636,2.100821,10.145962,12.429242,6.346375,9.049868,1.412779,1.344900,4.969902,6.005656,...,12.429242,12.314484,12.617578,12.611182,21.364939,12.314484,12.314484,12.617578,5.199363,7.537233
The Higher Path,16.966819,6.667834,10.903708,14.275754,9.518796,6.852942,7.274020,8.273840,9.798661,12.109904,...,14.275754,14.379368,11.296214,14.396604,26.313695,14.379368,14.379368,11.296214,12.984147,4.487084
Kushfly,13.221308,1.849670,10.251097,13.123732,7.082776,7.858424,2.127147,3.307870,6.357537,8.162358,...,13.123732,13.090254,12.097047,13.290388,23.448233,13.090254,13.090254,12.097047,8.099634,4.701482
MedMen West Hollywood,11.340067,1.806477,7.440296,10.361330,4.453535,5.366260,2.448041,2.903396,4.158496,6.362524,...,10.361330,10.344637,9.372360,10.524146,21.194969,10.344637,10.344637,9.372360,7.250996,6.905175


In [234]:
# for each marijuana arrest, number of dispensaries that are 0.5 miles or less 

arrest_distance['disp_0.5_mile'] = (distance_arrest_dispensary < 0.5).sum()

# for each marijuana arrest, number of dispensaries that is 1 mile or less 

arrest_distance['disp_1_mile'] = (distance_arrest_dispensary < 1).sum()

In [235]:
arrest_distance.head()

,Report ID,lat,long,Extra Special Delivery,Ganjarunner,The Higher Path,Kushfly,MedMen West Hollywood,Green Earth Collective,HERB,...,Purecannastore,Spectwonders,Venice Medical Center,Pasadena 419,Venice Kush,The Blue Diamond Center,101 CBD,Bud Cellar,disp_0.5_mile,disp_1_mile
0,151804330,33.9456,-118.2739,16.310674,10.873636,16.966819,13.221308,11.340067,12.719135,6.883323,...,7.958150,5.016343,8.882261,16.573949,2.699480,14.978295,66.809728,16.215627,1,2
1,4196822,34.1016,-118.3387,4.951664,2.100821,6.667834,1.849670,1.806477,7.479557,6.465254,...,10.019831,15.738283,6.276706,12.944058,9.842293,3.699940,58.722302,4.861954,0,5
2,4197433,33.9920,-118.4201,12.314997,10.145962,10.903708,10.251097,7.440296,15.007006,10.376742,...,13.837747,13.894923,2.604547,20.327679,9.802252,12.083558,57.950214,12.199388,1,2
3,4197611,33.9440,-118.4073,15.461070,12.429242,14.275754,13.123732,10.361330,16.692006,11.329906,...,14.218789,12.074944,5.566106,21.709643,9.353913,15.014068,60.337654,15.346250,0,1
4,4198423,34.0260,-118.3652,9.716038,6.346375,9.518796,7.082776,4.453535,11.079990,6.707892,...,10.396720,12.588260,1.797437,16.414634,7.350535,8.992752,59.549391,9.605556,0,1


I will now append `disp_0.5_mile` and `disp_1_mile` to `arrests`.

In [221]:
# Checking Report ID values are the same for each dataframe
len(arrest_distance['Report ID'].unique()) == len(arrests['Report ID'].unique())

True

In [281]:
arrests = arrests.merge(arrest_distance[['Report ID', 'disp_0.5_mile','disp_1_mile']], 
                        left_on ='Report ID', 
                        right_on = 'Report ID')

In [282]:
arrests.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,...,Charge,Charge Description,Address,Cross Street,Location,lat,long,marijuana_related,disp_0.5_mile,disp_1_mile
0,151804330,2015-01-01,1610.0,18,Southeast,1822,19,M,B,16.0,...,11357BHS,possess 28.5 grams or less of marijuana**,MAIN ST,CENTURY BL,"[33.9456, -118.2739]",33.9456,-118.2739,possess 28.5 grams or less of marijuana**,1,2
1,4196822,2015-01-01,820.0,6,Hollywood,645,23,M,B,16.0,...,11360(A)HS,transport/sell/furnish/etc marijuana,6800 HOLLYWOOD BL,NaN,"[34.1016, -118.3387]",34.1016,-118.3387,transport/sell/furnish/etc marijuana,0,5
2,4197433,2015-01-02,1030.0,14,Pacific,1457,24,F,W,16.0,...,11360(A)HS,transport/sell/furnish/etc marijuana,12300 CULVER BL,NaN,"[33.992, -118.4201]",33.9920,-118.4201,transport/sell/furnish/etc marijuana,1,2
3,4197611,2015-01-02,1530.0,14,Pacific,1494,30,M,O,16.0,...,11359HS,possession marijuana for sale,300 WORLD WY,NaN,"[33.944, -118.4073]",33.9440,-118.4073,possession marijuana for sale,0,1
4,4198423,2015-01-03,1940.0,3,Southwest,311,26,M,H,16.0,...,11360(A)HS,transport/sell/furnish/etc marijuana,JEFFERSON,COCHRAN,"[34.026, -118.3652]",34.0260,-118.3652,transport/sell/furnish/etc marijuana,0,1


### Distance between Arrests and Schools

I will now calculate the distance in miles between each arrest and each school.
There are 909 schools and 4156 marijuana related arrests. The process is identical to calculating the distance between the arrests and dispensaries.

In [254]:
school_distance = LA_pub_school[['School','lat','long']]
school_distance.head()

,School,lat,long
0,Frank Del Olmo Elementary,34.072925,-118.292591
1,10th Street Elementary,34.048317,-118.273235
2,Elementary Community Day School,34.251825,-118.563377
3,Cesar E Chavez Learning Academy - Arts/Theatre...,34.288514,-118.421011
4,San Pedro Senior High Gifted STEAM Magnet,33.730370,-118.299342


In [251]:
LA_pub_school.shape

(909, 9)

In [256]:
arrest_distance_school = arrests[['Report ID', 'lat', 'long']]
arrest_distance_school.reset_index(drop=True, inplace= True)
arrest_distance_school.head()

,Report ID,lat,long
0,151804330,33.9456,-118.2739
1,4196822,34.1016,-118.3387
2,4197433,33.9920,-118.4201
3,4197611,33.9440,-118.4073
4,4198423,34.0260,-118.3652


In [257]:
for school_name, position in zip(list(LA_pub_school['School'].values),range(len(LA_pub_school))):

    distance_list = []

    for arrest in range(len(arrest_distance_school)):
        distance = coord_dist(
                              LA_pub_school[LA_pub_school['School']== school_name]['lat'][position],
                              LA_pub_school[LA_pub_school['School']== school_name]['long'][position], 
                              arrest_distance_school['lat'][arrest], 
                              arrest_distance_school['long'][arrest])

        distance_list.append(distance)

    arrest_distance_school[school_name] = distance_list

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [259]:
arrest_distance_school.head()

,Report ID,lat,long,Frank Del Olmo Elementary,10th Street Elementary,Elementary Community Day School,Cesar E Chavez Learning Academy - Arts/Theatre/Entertain Mag,San Pedro Senior High Gifted STEAM Magnet,Normont Elementary,Stoney Point Continuation High,...,Hobart Boulevard Elementary,San Miguel Elementary,Wadsworth Avenue Elementary,Topeka Drive Charter for Advanced Studies,Madison Elementary,Stonehurst Avenue Elementary STEAM Magnet,Mariposa-Nabi Primary Center,99th Street Elementary,Pacoima Middle School,Alfred B Nobel Charter Middle School
0,151804330,33.9456,-118.2739,8.841093,7.079777,26.851213,25.094896,14.906111,10.455308,27.762139,...,7.639956,4.189603,4.494421,26.032771,3.433567,21.668439,7.549895,0.898472,22.658352,26.390100
1,4196822,34.1016,-118.3387,3.301209,5.252163,16.518848,13.718673,25.686054,21.315014,17.654500,...,3.828621,13.361318,7.832514,15.587916,12.910182,10.360802,4.038401,11.656371,11.314783,15.955269
2,4197433,33.9920,-118.4201,9.200422,9.279650,19.702017,20.437430,19.322782,15.441567,20.347550,...,7.788341,12.989126,9.347845,19.053126,12.252464,17.991424,8.115873,9.811876,17.780008,19.377917
3,4197611,33.9440,-118.4073,11.059560,10.532305,23.027052,23.758702,15.979518,12.215449,23.628284,...,9.509973,11.852757,9.658810,22.400379,11.096976,21.176960,9.748926,8.561424,21.107297,22.720570
4,4198423,34.0260,-118.3652,5.273880,5.496579,19.268434,18.374372,20.724342,16.474886,20.148632,...,3.879055,10.947515,6.220594,18.475587,10.291736,15.418625,4.222009,8.237573,15.790444,18.828595


I will now create two columns, `school_0.5_mile` and `school_1_mile`, that will count the number of schools that are within 0.5 miles and 1 mile respectively for each arrest. 

Creating a filter to append number of marijuana arrests within a half mile and 1 mile of a school.

In [265]:
distance_arrest_school = arrest_distance_school[LA_pub_school['School']].T
distance_arrest_school.head()

,0,1,2,3,4,5,6,7,8,9,...,4146,4147,4148,4149,4150,4151,4152,4153,4154,4155
Frank Del Olmo Elementary,8.841093,3.301209,9.200422,11.059560,5.273880,9.007413,2.780702,1.679027,3.623838,4.084471,...,11.059560,10.903239,11.967906,11.245559,19.360744,10.903239,10.903239,11.967906,3.224034,9.485727
10th Street Elementary,7.079777,5.252163,9.279650,10.532305,5.496579,9.981485,4.784401,3.633057,3.771729,2.941354,...,10.532305,10.321945,12.276535,10.718774,17.647277,10.321945,10.321945,12.276535,1.200118,11.512134
Elementary Community Day School,26.851213,16.518848,19.702017,23.027052,19.268434,15.415641,17.059912,18.145269,19.746271,22.084626,...,23.027052,23.213462,18.884459,23.105700,35.666862,23.213462,23.213462,18.884459,22.931271,11.376618
Cesar E Chavez Learning Academy - Arts/Theatre/Entertain Mag,25.094896,13.718673,20.437430,23.758702,18.374372,16.501221,13.969767,15.147259,18.074934,20.031908,...,23.758702,23.827644,20.938683,23.892658,35.177185,23.827644,23.827644,20.938683,19.772350,7.184009
San Pedro Senior High Gifted STEAM Magnet,14.906111,25.686054,19.322782,15.979518,20.724342,23.618696,25.634546,24.566260,21.156382,19.585208,...,15.979518,15.797416,20.413923,15.902609,4.448011,15.797416,15.797416,20.413923,21.095569,32.028419


In [389]:
# for each school, number of arrests that are within 0.5 miles or less 

school_distance['arrest_0.5_mile'] = school_distance['School'].map((distance_arrest_school<0.5).sum(1))

# for each school, number of arrests that is within 1 mile or less 

school_distance['arrest_1_mile'] = school_distance['School'].map((distance_arrest_school<1).sum(1))

In [390]:
school_distance.head()

,School,lat,long,arrest_0.5_mile,arrest_1_mile
0,Frank Del Olmo Elementary,34.072925,-118.292591,21,80
1,10th Street Elementary,34.048317,-118.273235,20,106
2,Elementary Community Day School,34.251825,-118.563377,0,2
3,Cesar E Chavez Learning Academy - Arts/Theatre...,34.288514,-118.421011,1,9
4,San Pedro Senior High Gifted STEAM Magnet,33.730370,-118.299342,14,63


In [338]:
list(school_distance['School']) == list(LA_pub_school['School'])

True

In [340]:
LA_pub_school = LA_pub_school.merge(school_distance[['School', 'arrest_0.5_mile','arrest_1_mile']], 
                                    left_on ='School', 
                                    right_on = 'School')

In [383]:
LA_pub_school = LA_pub_school.drop('coord',1)

LA_pub_school.head()

,School,Zip Code,Grades,Local District,Board District,complete_address,lat,long,arrest_0.5_mile,arrest_1_mile
0,Frank Del Olmo Elementary,90004,K- 5,C,2-Monica Garcia,100 N New Hampshire Ave Los Angeles CA 90004,34.072925,-118.292591,21,80
1,10th Street Elementary,90015,1- 5,C,2-Monica Garcia,1000 Grattan St Los Angeles CA 90015,34.048317,-118.273235,20,106
2,Elementary Community Day School,91311,K- 6,XS,3-Scott M Schmerelson,10001 Jumilla Ave Chatsworth CA 91311,34.251825,-118.563377,0,2
3,Cesar E Chavez Learning Academy - Arts/Theatre...,91340,9-12,NE,6-Kelly Gonez,1001 Arroyo Ave San Fernando CA 91340,34.288514,-118.421011,1,9
4,San Pedro Senior High Gifted STEAM Magnet,90731,9-11,S,7-Dr Richard A Vladovic,1001 W 15th St San Pedro CA 90731,33.730370,-118.299342,14,63


I will now append the number of schools within 0.5 miles and 1 mile for each arrest by creating two columns, `school_0.5_mile` and `school_1_mile`.

In [299]:
arrests.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,...,Charge,Charge Description,Address,Cross Street,Location,lat,long,marijuana_related,disp_0.5_mile,disp_1_mile
0,151804330,2015-01-01,1610.0,18,Southeast,1822,19,M,B,16.0,...,11357BHS,possess 28.5 grams or less of marijuana**,MAIN ST,CENTURY BL,"[33.9456, -118.2739]",33.9456,-118.2739,possess 28.5 grams or less of marijuana**,1,2
1,4196822,2015-01-01,820.0,6,Hollywood,645,23,M,B,16.0,...,11360(A)HS,transport/sell/furnish/etc marijuana,6800 HOLLYWOOD BL,NaN,"[34.1016, -118.3387]",34.1016,-118.3387,transport/sell/furnish/etc marijuana,0,5
2,4197433,2015-01-02,1030.0,14,Pacific,1457,24,F,W,16.0,...,11360(A)HS,transport/sell/furnish/etc marijuana,12300 CULVER BL,NaN,"[33.992, -118.4201]",33.9920,-118.4201,transport/sell/furnish/etc marijuana,1,2
3,4197611,2015-01-02,1530.0,14,Pacific,1494,30,M,O,16.0,...,11359HS,possession marijuana for sale,300 WORLD WY,NaN,"[33.944, -118.4073]",33.9440,-118.4073,possession marijuana for sale,0,1
4,4198423,2015-01-03,1940.0,3,Southwest,311,26,M,H,16.0,...,11360(A)HS,transport/sell/furnish/etc marijuana,JEFFERSON,COCHRAN,"[34.026, -118.3652]",34.0260,-118.3652,transport/sell/furnish/etc marijuana,0,1


In [369]:
arrests['school_0.5_mile'] = (distance_arrest_school<0.5).sum()
arrests['school_1_mile'] = (distance_arrest_school<1).sum()

In [349]:
arrests.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,...,Address,Cross Street,Location,lat,long,marijuana_related,disp_0.5_mile,disp_1_mile,school_0.5_mile,school_1_mile
0,151804330,2015-01-01,1610.0,18,Southeast,1822,19,M,B,16.0,...,MAIN ST,CENTURY BL,"[33.9456, -118.2739]",33.9456,-118.2739,possess 28.5 grams or less of marijuana**,1,2,5,5
1,4196822,2015-01-01,820.0,6,Hollywood,645,23,M,B,16.0,...,6800 HOLLYWOOD BL,NaN,"[34.1016, -118.3387]",34.1016,-118.3387,transport/sell/furnish/etc marijuana,0,5,2,2
2,4197433,2015-01-02,1030.0,14,Pacific,1457,24,F,W,16.0,...,12300 CULVER BL,NaN,"[33.992, -118.4201]",33.9920,-118.4201,transport/sell/furnish/etc marijuana,1,2,5,5
3,4197611,2015-01-02,1530.0,14,Pacific,1494,30,M,O,16.0,...,300 WORLD WY,NaN,"[33.944, -118.4073]",33.9440,-118.4073,possession marijuana for sale,0,1,0,0
4,4198423,2015-01-03,1940.0,3,Southwest,311,26,M,H,16.0,...,JEFFERSON,COCHRAN,"[34.026, -118.3652]",34.0260,-118.3652,transport/sell/furnish/etc marijuana,0,1,2,2


I will now remove variables from `arrests` that will not be useful for future exploratory data analysis and modeling purposes.

In [350]:
for column in arrests.columns:
    print(column)

Report ID
Arrest Date
Time
Area ID
Area Name
Reporting District
Age
Sex Code
Descent Code
Charge Group Code
Charge Group Description
Arrest Type Code
Charge
Charge Description
Address
Cross Street
Location
lat
long
marijuana_related
disp_0.5_mile
disp_1_mile
school_0.5_mile
school_1_mile


In [353]:
# One unique value, so it will be removed

arrests['Charge Group Description'].value_counts()

Narcotic Drug Laws    4156
Name: Charge Group Description, dtype: int64

`Area ID` is a numeric categorical representation of `Area Name` so it will be removed. The index already serves as a unique identifier, so the `Report ID` will be removed. `Reporting District`, `Charge Group Code`, `Charge Group Description`, `Charge`, `Address`, `Cross Street`, and `Location` will be removed. `marijuana_related` will be removed because it is a duplicate of the `Charge Description`.

In [364]:
list(arrests['marijuana_related']) == list(arrests['Charge Description'])

True

In [366]:
arrests = arrests.drop(['Report ID',
                        'Area ID',
                        'Reporting District',
                        'Charge Group Code',
                        'Charge Group Description',
                        'Charge',
                        'Address',
                        'Cross Street',
                        'Location',
                        'marijuana_related'
                        ],1)

In [370]:
arrests.head()

,Arrest Date,Time,Area Name,Age,Sex Code,Descent Code,Arrest Type Code,Charge Description,lat,long,disp_0.5_mile,disp_1_mile,school_0.5_mile,school_1_mile
0,2015-01-01,1610.0,Southeast,19,M,B,M,possess 28.5 grams or less of marijuana**,33.9456,-118.2739,1,2,5,16
1,2015-01-01,820.0,Hollywood,23,M,B,F,transport/sell/furnish/etc marijuana,34.1016,-118.3387,0,5,2,4
2,2015-01-02,1030.0,Pacific,24,F,W,F,transport/sell/furnish/etc marijuana,33.9920,-118.4201,1,2,5,9
3,2015-01-02,1530.0,Pacific,30,M,O,F,possession marijuana for sale,33.9440,-118.4073,0,1,0,0
4,2015-01-03,1940.0,Southwest,26,M,H,F,transport/sell/furnish/etc marijuana,34.0260,-118.3652,0,1,2,5


In [368]:
list(arrests['school_0.5_mile']) == list(arrests['school_1_mile'])

True

### Saving Dataframes:

Checking for nulls:

In [377]:
arrests.isnull().sum().sum()

0

In [384]:
LA_pub_school.isnull().sum().sum()

0

In [394]:
df_weed.isnull().sum().sum()

0

In [378]:
arrests.to_csv('../data/all_marijuana_arrests.csv', index = False)

In [387]:
LA_pub_school.to_csv('../data/LAUSD.csv', index = False)

In [397]:
df_weed.to_csv('../data/df_weed_updated.csv', index = False)

# Please go to Part 2B: Exploratory Data Analysis